<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/04_hsv-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-27 01:57:40--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12110 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.83K  --.-KB/s    in 0s      

2022-06-27 01:57:40 (66.7 MB/s) - ‘utils.py’ saved [12110/12110]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [3]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub


from utils import ModelCreator, get_train_val_ds, get_class_weight

In [4]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/feature_vector/2'

In [5]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [6]:
train_ds, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 17043 files belonging to 16 classes.
Found 3495 files belonging to 16 classes.


In [7]:
classes, class_weight = get_class_weight()

In [8]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

val_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tf.image.rgb_to_hsv(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [9]:
hsv_creator = ModelCreator(HUB_URL, 'HSV-model')
hsv_model = hsv_creator.make_model(img_size=IMG_SIZE)

Model: "HSV-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1536)              12930622  
                                                                 
 dense (Dense)               (None, 16)                24592     
                                                                 
Total params: 12,955,214
Trainable params: 12,845,998
Non-trainable params: 109,216
_________________________________________________________________


In [10]:
callbacks = hsv_creator.get_callbacks()

In [11]:
history = hsv_model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=15,
                        validation_data=val_ds,
                        class_weight=class_weight)

Epoch 1/15
533/533 [==============================] - 3256s 6s/step - loss: 20.7552 - auc: 0.6482 - accuracy: 0.5822 - val_loss: 1.3914 - val_auc: 0.5804 - val_accuracy: 0.5448
Epoch 2/15
533/533 [==============================] - 246s 459ms/step - loss: 12.3000 - auc: 0.8253 - accuracy: 0.7456 - val_loss: 1.1431 - val_auc: 0.6645 - val_accuracy: 0.6266
Epoch 3/15
533/533 [==============================] - 244s 455ms/step - loss: 9.3832 - auc: 0.8810 - accuracy: 0.7991 - val_loss: 1.0221 - val_auc: 0.7265 - val_accuracy: 0.6835
Epoch 4/15
533/533 [==============================] - 244s 456ms/step - loss: 7.0803 - auc: 0.9195 - accuracy: 0.8420 - val_loss: 1.0406 - val_auc: 0.7751 - val_accuracy: 0.7256
Epoch 5/15
533/533 [==============================] - 258s 482ms/step - loss: 5.7515 - auc: 0.9402 - accuracy: 0.8704 - val_loss: 1.0524 - val_auc: 0.7668 - val_accuracy: 0.7067
Epoch 6/15
533/533 [==============================] - 243s 454ms/step - loss: 4.1143 - auc: 0.9612 - accuracy:

In [12]:
history = hsv_model.fit(train_ds,
                          callbacks=callbacks,
                          epochs=20,
                          initial_epoch=15,
                          validation_data=val_ds,
                          class_weight=class_weight)

Epoch 16/20
533/533 [==============================] - 242s 452ms/step - loss: 0.3012 - auc: 0.9988 - accuracy: 0.9923 - val_loss: 1.4870 - val_auc: 0.7898 - val_accuracy: 0.7883
Epoch 17/20
533/533 [==============================] - 243s 454ms/step - loss: 0.3248 - auc: 0.9982 - accuracy: 0.9917 - val_loss: 1.3246 - val_auc: 0.8145 - val_accuracy: 0.7971
Epoch 18/20
533/533 [==============================] - 258s 482ms/step - loss: 0.2980 - auc: 0.9990 - accuracy: 0.9930 - val_loss: 1.4561 - val_auc: 0.7985 - val_accuracy: 0.7951
Epoch 19/20
533/533 [==============================] - 243s 454ms/step - loss: 0.2384 - auc: 0.9989 - accuracy: 0.9941 - val_loss: 1.4761 - val_auc: 0.7967 - val_accuracy: 0.8026
Epoch 20/20
533/533 [==============================] - 242s 453ms/step - loss: 0.2382 - auc: 0.9989 - accuracy: 0.9938 - val_loss: 1.5171 - val_auc: 0.7919 - val_accuracy: 0.7934


In [13]:
!cp ./logs/checkpoints/HSV-model.h5 ./drive/MyDrive/checkpoints/hsv-model.h5